In [1]:
import os
import pathlib
import random
import tensorflow as tf
import tensorflow_hub as tfhub

W0427 20:50:19.899661 140229051844480 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In [0]:
#Data
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 5
epochs= 10
steps_per_epoch = 10
summary_steps = 10

#Model
log_step = 10
train_batch_size = 5
eval_batch = 5
eval_delay_secs = 1 
throttle_secs = 1
train_steps = 1050

In [0]:
#tf.logging.info('TF Version {}'.format(tf.__version__))
#tf.logging.info('GPU Available {}'.format(tf.test.is_gpu_available()))
#if 'TF_CONFIG' in os.environ:
#    tf.logging.info('TF_CONFIG: {}'.format(os.environ["TF_CONFIG"]))

**Rutas**

In [0]:
# Modelo general
path_model = './model/'
os.makedirs(os.path.join(path_model), exist_ok=True)

# Modelo importado
TFHUB_CACHE_DIR = os.path.join(path_model, 'TFHub')
os.environ['TFHUB_CACHE_DIR'] = TFHUB_CACHE_DIR

# Data importada
path_data = os.path.join(path_model, 'datasets')

# Modelo importado
**Importacion**

In [0]:
classifier_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2" #@param {type:"string"}
tfh_module = tfhub.Module(classifier_url)

**Variables derivadas**

In [0]:
IMAGE_SIZE = tfhub.get_expected_image_size(tfh_module)

# Datos


## Adquisicion

In [0]:
url = 'http://files.fast.ai/data/dogscats.zip'
path_data = tf.keras.utils.get_file(fname = os.path.basename(url), 
                                    origin = url, 
                                    cache_dir=path_model,
                                    extract=True)
path_data = os.path.dirname(path_data)

**Directorios de datos**

In [0]:
train_path = os.path.join(data_directory, 'train')
train_files = os.path.join(data_directory, 'train', '**/*.jpg')
eval_path = os.path.join(data_directory, 'valid')
eval_files = os.path.join(data_directory, 'valid', '**/*.jpg')
pred_path = os.path.join(data_directory, 'sample/train')
pred_files = os.path.join(data_directory, 'sample/train','**/*.jpg')

In [8]:
pred_image = list(pathlib.Path(pred_path).glob('*/*'))
pred_image

[PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.4600.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.3570.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.4865.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.11737.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.394.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.2921.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.2266.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/cats/cat.9021.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.8091.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.8643.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.9077.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.1614.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.6768.jpg'),
 PosixPath('/tmp/datasets/dogscats/sample/train/dogs/dog.2423.jpg'),
 PosixPath('/tmp/datasets/dogscats

In [0]:
data_root = pathlib.Path(train_path)
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_to_index = dict((name, index) for index,name in enumerate(label_names))
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                  for path in all_image_paths]

---
For this output, you can connect additional layers according to the problem you want to solve. For example, if the problem is to classify images into 10 categories, you may want to continue fully connected-layer with 10 nodes, and apply the softmax function to have the probabilities.


logits = tf.layers.dense(inputs=outputs, units=10)
predictions = {
    "classes": tf.argmax(input=logits, axis=1),
    "probabilities": tf.nn.softmax(logits)
}

---
train_input_fn = tf.data.Dataset.list_files(train_files)

---
label = tf.string_split(source = [train_files], delimiter = '/').values[-2]

---
from IPython.core.debugger import Tracer; Tracer()() 
n, c, q

# Dataset

In [0]:
def read_dataset(filename, mode, batch_size):
  
  #Creacion de la ruta de las imagenes
  data_root = pathlib.Path(filename)
  all_image_paths = list(data_root.glob('*/*'))
  all_image_paths = [str(path) for path in all_image_paths]
  random.shuffle(all_image_paths)
  
  label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
  label_to_index = dict((name, index) for index,name in enumerate(label_names))
  all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]
  
  #Decodificar y cambiar el size de la imagen
  def _load_and_preprocess_image(path):
  #def _input_fn():
    def _preprocess_image(image):
      image = tf.image.decode_jpeg(image, channels=3)
      image = tf.image.resize(image, IMAGE_SIZE)
      image /= 255.9 # se esta normalizando a [0, 1]
      return image

    image_encoded = tf.read_file(path)
    image_decoded = _preprocess_image(image_encoded)

    return image_decoded
  #return _input_fn
  
  #Toma la ruta de las imagenes, la decodifica, crea el tensor de atributos
  paths_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
  image_ds = paths_ds.map(_load_and_preprocess_image)
  label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))
  image_count = len(all_image_paths)
  

  ds = tf.data.Dataset.zip((image_ds, label_ds))
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None #Loop indefinidamente
    ds = ds.shuffle(buffer_size = batch_size) # video 7:10 https://www.youtube.com/watch?v=uIcqeP7MFH0
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=10*batch_size)) #10*batch_size -- image_count
  else:
    num_epochs = 1 #end-of-input after this
  
  ds = ds.repeat(epochs).batch(BATCH_SIZE).prefetch(AUTOTUNE) # video 7:48
  return ds.make_one_shot_iterator().get_next()

In [0]:
def train_input_fn():
    return read_dataset(
        filename = train_path, #args['train_data_paths']
        batch_size = train_batch_size, #args['train_batch_size']
        mode = tf.estimator.ModeKeys.TRAIN)

def eval_input_fn():
  return read_dataset(
      filename = eval_path, #args['eval_data_paths']
      batch_size = eval_batch, 
      mode = tf.estimator.ModeKeys.EVAL)

def pred_input_fn():
  return read_dataset(
      filename = pred_path,
      batch_size = eval_batch, 
      mode = tf.estimator.ModeKeys.PRED)

def serving_input_fn():
  json_feature_placeholders = {
    'image' : tf.placeholder(tf.float32, [None]),}
  features = json_feature_placeholders
  return tf.estimator.export.ServingInputReceiver(features, json_feature_placeholders)

  

# Modelo
**Configuracion del modelo importado**

In [0]:
def model_fn(features, labels, mode, params):
  #module = hub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_050_192/feature_vector/2")
  #module = hub.Module("https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1")
  tfh_module=tfhub.Module("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/2")
  transformed_features = tfh_module(features)
  logits = tf.layers.dense(transformed_features, len(label_names))
  probabilities = tf.nn.softmax(logits)
  
  if (mode != tf.estimator.ModeKeys.PREDICT):
    one_hot_labels = tf.one_hot(labels, len(label_names))
    loss = tf.losses.softmax_cross_entropy(one_hot_labels, logits)
    optimizer = tf.train.AdamOptimizer() 
    train_op = tf.contrib.training.create_train_op(loss, optimizer)
    accuracy = tf.metrics.accuracy(labels, tf.argmax(probabilities, axis=-1))
    metrics = {'acc':accuracy}
  else:
    # None of these can be computed in prediction mode because labels are not available
    loss = optimizer = train_op = metrics = None
  
  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=train_op,
      # Nice: in estimator, you can return computed results that are not part of model training (here: class)
      predictions={'proba': probabilities, 'class': tf.argmax(probabilities, axis=-1)},
      eval_metric_ops=metrics
  )

# Entrenamiento

In [0]:
def train_and_evaluate(): #args
  #% 3 Estimador del modelo a usar
  run_config = tf.estimator.RunConfig(
    model_dir = "/tmp") #args['output_dir'],  Ouput directory for checkpoint
    #save_summary_steps = summary_steps, 
    #save_checkpoints_steps = steps_per_epoch, #, save_checkpoints_step = 100
    #log_step_count_steps=log_step)

  # Define los aspectos del modelo # ojo aqui esta el transfer learning
  model = tf.estimator.Estimator(
    model_fn = model_fn,
    config = run_config)

  # Define los aspectos del entrenamiento y la entrada de datos
  train_spec = tf.estimator.TrainSpec(
    input_fn = train_input_fn, 
    max_steps = train_steps) #args['train_steps']

  # Define los aspectos del uso en produccion con ML Engine
  export_latest = tf.estimator.LatestExporter(
    'exporter', #folder to export
    serving_input_receiver_fn = serving_input_fn)

  # Define los aspectos de la evaluacion, cada cuanto se graba para tensorboard y la entrada de datos
  eval_spec = tf.estimator.EvalSpec(
    input_fn = eval_input_fn)
    #steps = None,
    #start_delay_secs = eval_delay_secs, # args['eval_delay_secs'] start evaluating after N seconds
    #throttle_secs = throttle_secs) #, args['throttle_secs'] evaluate every N seconds
    #exporters = export_latest)

  # Ejecuta el modelo
  out = tf.estimator.train_and_evaluate(model, train_spec, eval_spec)
  return out

In [14]:
trained_model = train_and_evaluate()

INFO:tensorflow:Using config: {'_model_dir': '/tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb5484b6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0427 19:29:48.851213 140649632647040 estimator.py:201] Using config: {'_model_dir': '/tmp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7feb5484b6a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


W0427 19:29:48.855255 140649632647040 estimator.py:1924] Estimator's model_fn (<function model_fn at 0x7feb5484d730>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Not using Distribute Coordinator.


I0427 19:29:48.859803 140649632647040 estimator_training.py:185] Not using Distribute Coordinator.


...

I0427 19:34:44.281251 140649632647040 estimator.py:1979] Saving dict for global step 1050: acc = 0.986, global_step = 1050, loss = 0.032039676


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1050: /tmp/model.ckpt-1050


I0427 19:34:45.327067 140649632647040 estimator.py:2039] Saving 'checkpoint_path' summary for global step 1050: /tmp/model.ckpt-1050


INFO:tensorflow:Loss for final step: 0.1457856.


I0427 19:34:45.409554 140649632647040 estimator.py:359] Loss for final step: 0.1457856.
